In [1]:
import pandas as pd
import numpy as np

In [2]:
movie = pd.read_csv('tmdb_5000_movies.csv')
credit = pd.read_csv('tmdb_5000_credits.csv')

For each movie, compute the number of cast members

In [107]:
cast = pd.DataFrame()
cast_ = pd.DataFrame()
for index in credit.index:
    json = credit['cast'][index]
    movie_id = credit['movie_id'][index]
    title = credit['title'][index]
    cast_ = pd.read_json(json)
    cast_['movie_id'] = movie_id
    cast_['title'] = title
    concat =[cast, cast_]
    cast = pd.concat(concat)
number = cast.groupby(['movie_id', 'title']).count()['cast_id']

How many movies do not have a homepage?

In [3]:
len(movie['homepage']) - len(movie['homepage'].dropna())

3091

For each year, how many movies do not have a homepage?

In [4]:
moviey = movie[movie['homepage'].isnull()]
moviey['year'] = movie['release_date'].str.extract('(.*)-(.*)-(.*)')[0]
moviey = moviey.groupby('year').size()

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Extract the domain of each homepage.

In [283]:
import tldextract
movie_d = movie['homepage'].dropna()
movie_d = movie_d.to_frame()
movie_d['indice'] = -1
salva_doppi = pd.DataFrame()
salva_doppi['index'] = 0
conta = 0
for riga in movie_d.index:
    url = movie_d['homepage'][riga]
    url = url.split()
    if(len(url) != 1):
        movie_d['indice'][riga] = riga
        salva_doppi = salva_doppi.append(url)
        for split in range(conta,len(url)):
            salva_doppi['index'][split] = riga
        conta = conta + len(url)
salva_doppi = salva_doppi.rename(columns = {0: 'homepage'})
movie_d = movie_d.drop(movie_d[movie_d['indice'] != -1].index)
movie_d = movie_d.reset_index()
movie_d = movie_d.drop('indice', axis = 1)
movie_d = movie_d.append(salva_doppi)
movie_d = movie_d.reset_index().drop('level_0', axis = 1)
movie_d['domain'] = 0
for riga in movie_d.index:
    url = movie_d['homepage'][riga]
    movie_d['domain'][riga] = tldextract.extract(url).suffix
movie_t = movie['original_title']
movie_t = movie_t.to_frame()
movie_t = movie_t.reset_index()
movie_d = pd.merge(left = movie_d, right = movie_t, on = 'index')
movie_d = movie_d.set_index('index')

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\aless\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWar

Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary)